In [16]:
# Dependencies
import pandas as pd
import fastf1 as ff1
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from preprocessing import F1Session

In [2]:
# Enable fastf1 cache
ff1.Cache.enable_cache('cache')

In [3]:
training_races = [
    {'year': 2022, 'circuit': 1, 'session_type': 'R'},
    {'year': 2022, 'circuit': 2, 'session_type': 'R'},
    {'year': 2022, 'circuit': 3, 'session_type': 'R'},
    {'year': 2022, 'circuit': 4, 'session_type': 'R'},
    {'year': 2022, 'circuit': 5, 'session_type': 'R'},
    {'year': 2022, 'circuit': 6, 'session_type': 'R'},
    {'year': 2022, 'circuit': 7, 'session_type': 'R'},
    {'year': 2022, 'circuit': 8, 'session_type': 'R'},
    {'year': 2022, 'circuit': 9, 'session_type': 'R'},
    {'year': 2022, 'circuit': 10, 'session_type': 'R'},
    {'year': 2022, 'circuit': 11, 'session_type': 'R'},
    {'year': 2022, 'circuit': 12, 'session_type': 'R'},
    {'year': 2022, 'circuit': 13, 'session_type': 'R'},
    {'year': 2022, 'circuit': 14, 'session_type': 'R'},
    {'year': 2022, 'circuit': 15, 'session_type': 'R'},
    {'year': 2022, 'circuit': 16, 'session_type': 'R'},
    {'year': 2022, 'circuit': 17, 'session_type': 'R'},
    {'year': 2022, 'circuit': 18, 'session_type': 'R'},
    {'year': 2022, 'circuit': 19, 'session_type': 'R'},
    {'year': 2022, 'circuit': 20, 'session_type': 'R'},
    {'year': 2022, 'circuit': 21, 'session_type': 'R'},
    {'year': 2022, 'circuit': 22, 'session_type': 'R'},
]

testing_races = training_races = [
    {'year': 2023, 'circuit': 1, 'session_type': 'R'},
    {'year': 2023, 'circuit': 2, 'session_type': 'R'},
    {'year': 2023, 'circuit': 3, 'session_type': 'R'},
    {'year': 2023, 'circuit': 4, 'session_type': 'R'},
    {'year': 2023, 'circuit': 5, 'session_type': 'R'},
    {'year': 2023, 'circuit': 6, 'session_type': 'R'},
    {'year': 2023, 'circuit': 7, 'session_type': 'R'},
    {'year': 2023, 'circuit': 8, 'session_type': 'R'},
]

In [4]:
# Load the races into a session
training_session = F1Session(training_races)

# Iterate through the session to get the preprocessed data for each race
data_list = [
    preprocessed_data for preprocessed_data in training_session if preprocessed_data is not None]

# Concatenate all the dataframes into a single dataframe
training_data = pd.concat(data_list, ignore_index=True)

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.0.4]
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '55', '44', '18', '63', '77', '10', '23', '22', '2', '20', '21', '27', '24', '4', '31', '16', '81']
core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.0.4]
req            INFO 	Using cached data for driver_info

In [5]:
# Load the races into a session
testing_session = F1Session(testing_races)

# Iterate through the session to get the preprocessed data for each race
data_list = [
    preprocessed_data for preprocessed_data in testing_session if preprocessed_data is not None]

# Concatenate all the dataframes into a single dataframe
testing_data = pd.concat(data_list, ignore_index=True)

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.0.4]
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count


req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '55', '44', '18', '63', '77', '10', '23', '22', '2', '20', '21', '27', '24', '4', '31', '16', '81']
core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.0.4]
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for timing_data
req        

In [33]:
from torch import FloatTensor

class F1Dataset(Dataset):
    def __init__(self, data):
        self.X = data.drop('Podium', axis=1).values
        self.y = data['Podium'].values

    def __getitem__(self, idx):
        return FloatTensor(self.X[idx]), FloatTensor([self.y[idx]])

    def __len__(self):
        return len(self.X)


In [34]:
from torch.utils.data import DataLoader

train_dataset = F1Dataset(training_data)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = F1Dataset(testing_data)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [35]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


Using cpu device


In [36]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


In [38]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [39]:
learning_rate = 1e-3
epochs = 5

model = NeuralNetwork()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.